In [1]:
from openai import OpenAI

# Init DeepSeek API client
client = OpenAI(
    api_key="sk-f47fa990b47b47909bd1de489753fd8a", 
    base_url="https://api.deepseek.com"
)

In [9]:
def predict_sentiment_deepseek(text):
    prompt = f"""
You are a sentiment analysis expert. Classify the following social media post as one of the following sentiment categories:
- Positive
- Negative
- Neutral
- Mixed (e.g., it includes both positive and negative expressions. Example: Упаковка яркая и аккуратная, все материалы качественные, ничего не пролилось и не сломалось. Несмотря на то, что инструкции могли бы быть чуть более подробными, сам процесс оказался очень увлекательным. Особенно понравились ароматические добавки — свечи получились не только красивыми, но и приятно пахнущими. Ребёнок в восторге, сразу захотела делать ещё. Если ищете подарок для творческой девочки — очень рекомендую!)

Only return one of the labels: Positive, Negative, Neutral, or Mixed.

Text: "{text}"

Label:"""

    try:
        response = client.chat.completions.create(
            model="deepseek-chat",
            messages=[
                {"role": "system", "content": "You are a sentiment analysis assistant."},
                {"role": "user", "content": prompt}
            ]
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"ERROR: {e}"

In [3]:
import pandas as pd
df=pd.read_csv("ua_sentiment_dataset_with_test.csv")

In [4]:
texts=df["document_content"].values.tolist()

In [31]:
from tqdm import tqdm
# sentiments = []

for text in tqdm(texts[9016:]):  # Or texts[4876:] if continuing
    label = predict_sentiment_deepseek(text)
    sentiments.append(label)



100%|█████████████████████████████████████████| 269/269 [22:03<00:00,  4.92s/it]


In [37]:
sents=['Negative', 'Neutral','Positive','Mixed']
sentiment_cleaned=[]
for e in sentiments:
    if e not in sents:
        print('-',e)
        if 'Mixed' in e:
            sentiment_cleaned.append('Mixed')
        elif 'Neutral' in e:
            sentiment_cleaned.append('Neutral')
        elif 'Positive' in e:
            sentiment_cleaned.append('Positive')
        elif 'Negative' in e:
            sentiment_cleaned.append('Negative')
        else:
            sentiment_cleaned.append('Neutral')
    else:
        sentiment_cleaned.append(e)

- The sentiment of the given text can be classified as **Positive**. 

The statement conveys a proactive and optimistic message about efforts to increase the number of Patriot systems in Ukraine, which aligns with a positive sentiment. The emoji (👹) might seem out of place, but it doesn't negate the overall positive tone of the statement itself. 

Label: **Positive**
- The sentiment of the provided social media post can be classified as **Neutral**. The text primarily presents factual information about the history, construction, and features of St. Andrew's Church in Kyiv without expressing strong positive or negative emotions. While there is admiration implied in describing it as a "перлина Києва" (pearl of Kyiv), the overall tone remains informative rather than emotionally charged.  

**Label: Neutral**
- 
- 


In [38]:
set(sentiment_cleaned)

{'Mixed', 'Negative', 'Neutral', 'Positive'}

In [50]:
df=pd.read_csv("ua_sentiment_dataset_labeled_lang.csv")

In [51]:
df["sentiment_deepseek"] = sentiment_cleaned
df["annotator_response"] = df["annotator_response"].str.strip().str.lower()
df["sentiment_deepseek"] = df["sentiment_deepseek"].str.strip().str.lower()
df_filtered = df[df["annotator_response"].str.lower() != "idk"].copy()


In [52]:
[[df_filtered['language'].values.tolist().count(e),e] for e in list(set(df_filtered['language'].values.tolist()))]

[[4312, 'Code-mixed'], [1799, 'Russian'], [2924, 'Ukrainian']]

In [53]:
from sklearn.metrics import precision_recall_fscore_support, classification_report
import pandas as pd

def evaluate_sentiment(df, y_true_col, y_pred_col, group_col="language"):
    """
    Evaluate sentiment classification with overall and per-language-group metrics.
    
    Params:
    - df: pd.DataFrame containing predictions and true labels
    - y_true_col: column name of true labels (e.g. human annotations)
    - y_pred_col: column name of model predictions (e.g. DeepSeek output)
    - group_col: column to group by (e.g. 'language')

    Returns:
    - dict with overall metrics and per-group metrics
    """
    y_true = df[y_true_col]
    y_pred = df[y_pred_col]

    # Overall metrics
    overall_macro = precision_recall_fscore_support(y_true, y_pred, average='macro', zero_division=0)
    overall_micro = precision_recall_fscore_support(y_true, y_pred, average='micro', zero_division=0)

    result = {
        "overall": {
            "macro": {
                "precision": overall_macro[0],
                "recall": overall_macro[1],
                "f1": overall_macro[2],
            },
            "micro": {
                "precision": overall_micro[0],
                "recall": overall_micro[1],
                "f1": overall_micro[2],
            }
        },
        "by_group": {}
    }

    # Per-language group metrics
    for group_value in df[group_col].unique():
        subset = df[df[group_col] == group_value]
        if subset.empty:
            continue

        group_true = subset[y_true_col]
        group_pred = subset[y_pred_col]

        macro = precision_recall_fscore_support(group_true, group_pred, average='macro', zero_division=0)
        micro = precision_recall_fscore_support(group_true, group_pred, average='micro', zero_division=0)

        result["by_group"][group_value] = {
            "macro": {
                "precision": macro[0],
                "recall": macro[1],
                "f1": macro[2],
            },
            "micro": {
                "precision": micro[0],
                "recall": micro[1],
                "f1": micro[2],
            }
        }

    return result

metrics = evaluate_sentiment(df_filtered, y_true_col="annotator_response", y_pred_col="sentiment_deepseek")

import pprint
pprint.pprint(metrics)

{'by_group': {'Code-mixed': {'macro': {'f1': 0.5679725039763073,
                                       'precision': 0.6231562657145774,
                                       'recall': 0.5744749801144564},
                             'micro': {'f1': 0.6565398886827458,
                                       'precision': 0.6565398886827458,
                                       'recall': 0.6565398886827458}},
              'Russian': {'macro': {'f1': 0.43240327243491333,
                                    'precision': 0.47080404431791834,
                                    'recall': 0.4653637585769277},
                          'micro': {'f1': 0.5347415230683713,
                                    'precision': 0.5347415230683713,
                                    'recall': 0.5347415230683713}},
              'Ukrainian': {'macro': {'f1': 0.5398748021951669,
                                      'precision': 0.5486586215507706,
                                      'recall': 0.6

In [54]:
df.to_csv("ua_sentiment_dataset_labeled_lang_deepseek.csv", index=False)